In [4]:
search_for = 124
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004112720489501953
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 75
all cases: 621252555
type: [1, 1, 1, 1, 124] 124
cases of this type: 236421376
100000 30.27849453715281
200000 44.66864051123344
300000 43.457254942031355
400000 43.45266429697113
500000 45.00674552410228
600000 43.740513975968724
700000 42.91715896577568
800000 42.97425777468569
900000 42.562497228961334
1000000 15.367227646155731
1100000 43.788680320961674
1200000 42.51744674045984
1300000 41.75594460231356
1400000 42.48926040305905
1500000 44.722009869720544
1600000 42.6647503215125
1700000 42.6467008332005
1800000 42.93535248601913
1900000 37.40932740093278
2000000 21.528474083614284
2100000 41.9244032777264
2200000 42.210134308904045
2300000 41.0497680937986
2400000 42.768921470868044
2500000 42.09025831780037
2600000 40.852727914115775
2700000 41.574650512019005
2800000 41.09657601254313
2900000 17.48712772043787
3000000 40.158286852

29700000 41.458805479518276
29800000 40.29491114874286
29900000 40.94886406577916
30000000 42.10701147506645
30100000 43.06679956224236
30200000 44.02549839140669
30300000 26.88763383903706
30400000 38.35252846146831
30500000 44.27992220694878
30600000 42.69003859586893
30700000 42.66784330588633
30800000 19.890052482064004
30900000 43.22761391572238
31000000 43.306693762713216
31100000 42.05350224502329
31200000 42.03304738423106
31300000 38.1580301497429
31400000 38.19460487559605
31500000 16.743265793965865
31600000 39.247212076322356
31700000 38.0747705785258
31800000 40.30838472692301
31900000 40.48939891646984
32000000 43.624619132860225
32100000 43.10140556194335
32200000 23.709420360441555
32300000 41.077690784907524
32400000 42.90346052577458
32500000 44.634885379282245
32600000 43.387571239358614
32700000 26.079883680716367
32800000 37.75908204850344
32900000 44.14643887833528
33000000 42.092995100452605
33100000 41.4619192335778
33200000 40.18082755244971
33300000 38.2210261

59700000 30.358353499405446
59800000 37.32255040432359
59900000 36.315872634037945
60000000 38.05855377232671
60100000 14.734706106898388
60200000 37.84695327555363
60300000 36.85017265989839
60400000 37.276866152047425
60500000 36.8647545332907
60600000 19.75761129678017
60700000 39.225470601620614
60800000 40.579330732113036
60900000 40.51012656637067
61000000 43.06549319221319
61100000 40.35629232178427
61200000 38.586351637509246
61300000 38.52730406012607
61400000 37.908255129634966
61500000 31.34995744318114
61600000 23.866208683636625
61700000 34.43257757530046
61800000 36.57958702802494
61900000 34.311698278484585
62000000 17.083893160491254
62100000 33.580810022763025
62200000 36.17813122915324
62300000 33.56868098252676
62400000 35.35865146481165
62500000 21.022367831374556
62600000 36.99369621708051
62700000 37.572599731838345
62800000 38.60926368747858
62900000 38.72776617128287
63000000 40.305592096983176
63100000 40.275347761389206
63200000 40.73931776173243
63300000 38.5

89700000 37.86014795979708
89800000 38.763184565763275
89900000 36.15739684810366
90000000 34.97782650120136
90100000 34.97434084780388
90200000 34.90572968122578
90300000 34.01442760529366
90400000 16.497789403847015
90500000 32.00799173821018
90600000 15.491092977018027
90700000 31.935734538747308
90800000 26.644263539767373
90900000 26.07811022233796
91000000 35.31368703364156
91100000 33.93156463844191
91200000 34.87260879996402
91300000 35.315182001100645
91400000 37.09530186166311
91500000 37.99256080267757
91600000 37.137393226012186
91700000 37.15846598408536
91800000 35.418654787812955
91900000 32.730901317312714
92000000 34.15405106078696
92100000 34.166384044940315
92200000 33.27556595487978
92300000 15.849931253813498
92400000 34.35361029872134
92500000 16.301845995188973
92600000 30.3529771369861
92700000 23.00329061435485
92800000 27.74536253723733
92900000 32.93218515209921
93000000 34.223522764857385
93100000 33.3938379999445
93200000 33.561663707683735
93300000 36.0517

119000000 3.716146395173685
119100000 3.728442283935954
119200000 3.7215156938837435
119300000 3.719157826407754
119400000 3.7172379938386677
119500000 3.716574051369313
119600000 3.696912092481232
119700000 3.72681638268123
119800000 3.743701468629509
119900000 3.719282295454227
120000000 3.7546457807951077
120100000 3.7123916095823177
120200000 3.7161651084871687
120300000 3.7121717808751167
120400000 3.684430916272465
120500000 3.7047398664732314
120600000 3.6876526814088324
120700000 3.711044879208667
120800000 3.697741709807806
120900000 3.6817733778646886
121000000 3.7317741261531943
121100000 3.6945071768677398
121200000 3.6984221258364025
121300000 3.71026221017864
121400000 3.7066552735046745
121500000 3.6901220071611904
121600000 3.6947929509886768
121700000 3.698082455182135
121800000 3.7218571869117416
121900000 3.853669739139087
122000000 3.8695580594054824
122100000 3.8184541649674473
122200000 3.736107217099753
122300000 3.70019347047921
122400000 3.6965832148039843
1225

...

493000000 4.267222427225305
493100000 4.610292316030396
493200000 4.888403160163472
493300000 4.331079881111323
493400000 1.4714617358063888
493500000 3.658375707459102
493600000 3.2859686241416406
493700000 2.9565062007665968
493800000 3.895238514205456
493900000 4.536669785348498
494000000 4.103379345149702
494100000 4.809078822698086
494200000 3.956838074792375
494300000 2.6270521096726793
494400000 3.227212863005638
494500000 3.7484484579130903
494600000 2.548231646733751
494700000 3.7955059292474984
494800000 3.9399996932823362
494900000 4.158502071292128
495000000 4.826433828082512
495100000 4.516234904893875
495200000 4.174481996065676
495300000 1.2601510463271637
495400000 3.6690022047761675
495500000 2.3152044608445927
495600000 4.5935141825656
495700000 3.9882107548877963
495800000 3.9377484966069765
495900000 3.819586207625568
496000000 3.9606351604442533
496100000 5.119396771670984
496200000 2.672037390676558
496300000 2.5769441824634636
496400000 2.7655612318911253
4965000

521000000 3.3741109227827955
521100000 3.6655396364873254
521200000 3.0756149585469132
521300000 3.0466988501099714
521400000 3.124765436937541
521500000 3.1887740870596764
521600000 3.8328125235568185
521700000 3.0468468486612146
521800000 3.2362207992460696
521900000 3.26505147636404
522000000 3.357986162631184
522100000 3.7351089623442824
522200000 3.23064867580542
522300000 3.260496972893348
522400000 3.3532055412237947
522500000 3.3320880437274045
522600000 3.609286029638052
522700000 3.252158634546373
522800000 3.351643099513776
522900000 3.23969156866622
523000000 3.3477926699625784
523100000 3.4674743214594788
523200000 3.2966359181783895
523300000 3.194917910086221
523400000 3.157212124462293
523500000 3.101276200382531
523600000 3.2837679112079874
523700000 2.9427707096289923
523800000 3.1027361861768266
523900000 3.0955236823600267
524000000 3.1361939180054237
524100000 3.536549332912833
524200000 3.208547538629883
524300000 2.9936138095854754
524400000 3.719993105210423
524

547800000 3.9084386318901485
547900000 4.028762145588662
548000000 2.0805485634145207
548100000 3.2555573808128835
548200000 4.040989055258238
548300000 1.2516385048952268
548400000 0.5438156843267182
548500000 0.5420749857292706
548600000 0.5436578344344629
548700000 0.5403547838390768
548800000 0.5414761304031743
548900000 0.5429658881895575
549000000 0.5389492315793931
549100000 0.5383380604564051
549200000 0.540058997147835
549300000 0.5424012861227989
549400000 0.5458849935128622
549500000 0.5463747032402388
549600000 0.5435630466550887
549700000 0.5436022038360238
549800000 0.5354856484130621
549900000 0.537392325242877
550000000 0.532233684379879
550100000 0.5889981424290074
550200000 0.7143033008626302
550300000 0.6994123860018948
550400000 0.580311569179684
550500000 0.5259446860406398
550600000 0.5466313867619302
550700000 4.325378323928168
550800000 4.0428718280166684
550900000 1.9006524967289402
551000000 4.207082016113331
551100000 4.229641208083709
551200000 3.60493639089

576100000 1.059553756252408
576200000 1.054446318142328
576300000 1.536086787204345
576400000 1.5576046225241122
576500000 1.218005188383715
576600000 0.8080024992303252
576700000 1.3023958710979355
576800000 1.5457708373350039
576900000 1.145046749126007
577000000 1.5278725540272926
577100000 0.7150636937017507
577200000 1.525531194891294
577300000 1.5389023742022447
577400000 1.098003997444249
577500000 1.272662836272329
577600000 0.8431195991459257
577700000 1.615328564227147
577800000 1.2924411944659648
577900000 1.1959293459631537
578000000 1.0762217616560392
578100000 0.9945790675731798
578200000 1.5931242703241209
578300000 0.9656260504561761
578400000 1.341086836446643
578500000 0.7762347260484266
578600000 1.1992162292730544
578700000 1.4467946210242508
best so far: 0
type: [1, 2, 1, 31, 2] 124
cases of this type: 953312
578800000 0.4187448534907798
578900000 0.2968453099499875
579000000 0.28475076176320513
579100000 0.28322183652847344
579200000 0.27600564641123676
579300000 

599500000 0.15235154034853976
599600000 0.15415142129828532
599700000 0.15355370760603745
599800000 0.4056304791029162
599900000 0.7431770165272389
600000000 0.6290830677130222
600100000 0.4566359427949521
600200000 0.5098030720050931
600300000 0.6284749892394642
600400000 0.34848183595536814
600500000 0.7767650063798825
600600000 0.5616698815916776
600700000 0.5026018358324732
600800000 0.41550256927341556
600900000 0.29629469346860055
601000000 0.39752529632497163
601100000 0.5041598165340423
601200000 0.40907743974380695
601300000 0.48172333156607217
601400000 0.37659568607329325
601500000 0.6441578158543011
601600000 0.3634326953267091
601700000 0.7135792119240032
601800000 0.5519469706416429
601900000 0.5684127620262801
602000000 0.5108257277140253
602100000 0.2921282920455436
602200000 0.6222708344839646
602300000 0.4773460591968397
602400000 0.38130094779223206
602500000 0.5740001551022131
602600000 0.4613980951152477
602700000 0.6903060127280354
602800000 0.547753107271095
6029